In [1]:
!pip install transformers

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import string
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df = pd.read_csv('test.csv')
test_df.head()

In [ ]:
test_df.info()

In [ ]:
#funciones auxiliares
import re
regex = re.compile('[^a-zA-Z]')
def getPunctuationSigns(words):
    return ' '.join([item for item in words if len(regex.sub('', item)) == 0])

def removePunctuationSigns(words):
    return [item for item in words if len(regex.sub('', item)) > 0]


def getHashtags(words):
    return ' '.join([item.lstrip('#') for item in words if item.startswith('#') and len(item) > 1])

def getMentions(words):
    return ' '.join([item.lstrip('@') for item in words if item.startswith('@') and len(item) > 1])

def getURLs(words):
    return ' '.join([item for item in words if item.startswith('http')])

def clean(text):
    
    exclude = set(string.punctuation)
    clean_text = ''.join(ch for ch in text if ch not in exclude)
    
    exclude = set(stopwords.words('english'))
    clean_text_list = clean_text.split(' ')
    clean_text = ' '.join(ch for ch in clean_text_list if ch not in exclude)

    return clean_text

#test_df['clean_text'] = test_df['text'].apply(clean)
#train_df['clean_text'] = train_df['text'].apply(clean)
#train_df.head()

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


#Si tira error porque no lo reconoce ejecutar el siguiente codigo
import nltk
nltk.download('wordnet')

#Si tira error por no encontrar el 'averaged_perceptron_tagger' ejecutar el siguiente codigo
#nltk.download('averaged_perceptron_tagger')

def getWordNetPOSTag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    lemmaWords = []
    for w in text.split(" "):
        #lemmaWords.append(lemmatizer.lemmatize(w, getWordNetPOSTag(w)))
        lemmaWords.append(lemmatizer.lemmatize(w, pos='a')) 
        
    return " ".join(lemmaWords)

def porterStemmer(text):
    porter = PorterStemmer()
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

def snowballStemmer(text):
    porter = SnowballStemmer("english")
    stemmedWords = []
    for w in text.split(" "):
        stemmedWords.append(porter.stem(w)) 
        
    return " ".join(stemmedWords)

In [ ]:
#FEATURES
stop = stopwords.words('english')

def crear_features(df):
    
    #texto
    df['clean_text'] = df['text'].apply(clean)
    df['lemma_text'] = df['clean_text'].apply(lemmatizer)
    df['porter_stemmed_text'] = df['clean_text'].apply(porterStemmer)
    df['snowball_stemmed_text'] = df['clean_text'].apply(snowballStemmer)
    
    #categóricas
    df['words'] = df['text'].apply(lambda x: x.split(' '))
    df['hashtags'] = df['words'].apply(getHashtags) #Obtengo los hashtags: "ht1 ht2 ht3 ..."
    df['mentions'] = df['words'].apply(getMentions) #Obtengo las menciones: "men1 men2 men3 ..."
    df['urls'] = df['words'].apply(getURLs) #Obtengo las urls "url1 url2 url3 ..."
    df['stop_words'] = df['text'].apply(lambda x: [w for w in str(x).lower().split() if w in stop])
    df['real_words'] = df['words'].apply(removePunctuationSigns)
    
    #numéricas
    df['words_count'] = df['words'].apply(lambda x: len(x))
    df['character_count'] = df['text'].str.len()
    df['mean_word_length'] = df['text'].apply(lambda x: (sum(len(w) for w in str(x).split()) / len(str(x).split())))
    df['punctuation_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    df['stop_words_count'] = df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop]))
    

In [ ]:
crear_features(train_df)
crear_features(test_df)
train_df.head()

In [ ]:
def trp(l, n):
    return l[:n] + [0]*(n-len(l))


In [ ]:
def tokenizacion_de_texto(texto,longitud):
    marked_text = "[CLS] " + texto + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    vector = trp(indexed_tokens,longitud)
    return vector
    #return indexed_tokens

In [ ]:
train_df = train_df.fillna('')
test_df = test_df.fillna('')

In [ ]:
#Parametros
vocab_size = 30522
epochs = 3
maxlen = 140
n_words = 500
test_size = 0.25
padding_texto = 60

In [ ]:
#tokenización
def tokenizar_df(df):

    df['texto_tokenizado'] = df['text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['lemma_text_tokenizado'] = df['lemma_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['porter_stemmed_text_tokenizado'] = df['porter_stemmed_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['snowball_stemmed_text_tokenizado'] = df['snowball_stemmed_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['clean_text_tokenizado'] = df['clean_text'].apply(lambda x: tokenizacion_de_texto(x,padding_texto))
    df['keyword_tokenizado'] = df['keyword'].apply(lambda x: tokenizacion_de_texto(x,5))
    df['location_tokenizado'] = df['location'].apply(lambda x: tokenizacion_de_texto(x,7))
    df['mentions_tokenizado'] = df['mentions'].apply(lambda x: tokenizacion_de_texto(x,8))
    df['hashtags_tokenizado'] = df['hashtags'].apply(lambda x: tokenizacion_de_texto(x,8))
    df['real_words_tokenizado'] = df['real_words'].apply(lambda x: tokenizacion_de_texto(' '.join(x),8))

    df['words_count'] = df['words_count'].apply(lambda x: [x])
    df['character_count'] = df['character_count'].apply(lambda x: [x])
    df['mean_word_length'] = df['mean_word_length'].apply(lambda x: [x])
    df['punctuation_count'] = df['punctuation_count'].apply(lambda x: [x])
    df['stop_words_count'] = df['stop_words_count'].apply(lambda x: [x])
    
    df['features_tokenizados'] = (  #TEXTO
                                    df['texto_tokenizado']
                                    #df['clean_text_tokenizado']
                                    #df['lemma_text_tokenizado']
                                    #df['porter_stemmed_text_tokenizado']
                                    #df['snowball_stemmed_text_tokenizado']
                                    +df['real_words_tokenizado']
        
                                    #CATEGORICAS
                                    #+ df['keyword_tokenizado']
                                    #+ df['location_tokenizado']
                                    + df['mentions_tokenizado']
                                    + df['hashtags_tokenizado']
                                  
                                    #NUMERICAS
                                    #+ df['mean_word_length']
                                    #+ df['stop_words_count']
                                    #+ df['punctuation_count']
                                    )

In [ ]:
tokenizar_df(train_df)
tokenizar_df(test_df)
train_df.head()

In [ ]:
#split
FX, FY = train_df['features_tokenizados'], train_df['target']

X_train, X_test, y_train, y_test = train_test_split(train_df['features_tokenizados'], train_df['target'], test_size=test_size,
                                                  random_state=42)
FTest = test_df['features_tokenizados']
test_ids = test_df['id'] 

In [ ]:
X_train = np.array([np.array(lista) for lista in X_train])
X_a = np.array([np.array(lista) for lista in X_test])

padded_FX =   np.array([np.array(lista) for lista in FX])
padded_FY =   np.array([np.array(lista) for lista in FY])

padded_FTest = X_test = np.array([np.array(lista) for lista in FTest])
padded_train = pad_sequences(X_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(X_a, maxlen = maxlen, truncating = 'post')


In [ ]:
#secuencia_train = tokenizar(X_train.values)
#secuencia_test = tokenizar(X_test.values)
#secuencia_FTest = tokenizar(FTest.values)

#padded_train = pad_sequences(secuencia_train, maxlen = maxlen, truncating = 'post')
#padded_test = pad_sequences(secuencia_test, maxlen = maxlen)
#padded_FTest = pad_sequences(secuencia_FTest, maxlen = maxlen)

In [ ]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length= maxlen),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv1D(256, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
#model.fit(padded_FX, padded_FY, epochs=2)
model.fit(padded_train,y_train,epochs=epochs, validation_data=(padded_test, y_test))

In [ ]:
#validación del f1 score con el set de test
test_val = pd.read_csv('submission.csv')
#test_val.head()
y_FTest = test_val['target']
preds = model.predict_classes(padded_FTest)
print("F1 score:", f1_score(y_FTest, preds))

In [ ]:
preds_locales = model.predict_classes(padded_test)
preds_locales = pd.Series(list((x[0] for x in preds_locales)))
#preds_locales

In [ ]:
print("F1 score:", f1_score(y_test, preds_locales))

In [ ]:
#preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-Conv1D-features.csv')
df_preds.head()